# TFlite model maker api

In this colab we'll go over


1.   Training/validation CSVs [ from iNaturalist 2017 ]
2.   Training a tflite model
3.   Saving the model
4.   Running inference


In [ ]:
#%%
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#%%
import matplotlib
import matplotlib.pyplot as plt
import os
os.chdir('/home/actmsofficial/models/research/')
import random
import io
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
import json

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
#from object_detection.utils import colab_utils
#from object_detection.builders import model_builder
#from object_detection.utils import dataset_util, label_map_util
#%%

#in1 = open('/home/actmsofficial/Documents/annotations/iNat2021/train.json','r')
in1 = open('/home/actmsofficial/Documents/annotations/iNat2021/train.json','r')
out = json.load(in1)
in1.close()
imNames = out['images']
finalHyms = []
classDic1 = {}
fCat = ['Bombus','Apis','Formicidae','Nymphalidae','Muscidae','Tortricidae',
'Vespidae']
for ele in fCat:
  classDic1[ele] = []

for ele in imNames:
  if ele['file_name'].find('Insecta') != -1:
    finalHyms.append(ele['file_name'])
    for cat1 in fCat:
      if ele['file_name'].find(cat1) != -1:
        classDic1[cat1].append(ele['file_name'])


# %%
import json
with open('/home/actmsofficial/Documents/autopolls/tflite_training/train_2017_bboxes.json', "r") as f:
    annotations = json.load(f)

#with open('/home/actmsofficial/Documents/autopolls/tflite_training/val_2017_bboxes.json', "r") as f:
#    annotations = json.load(f)

images_dir = '/home/actmsofficial/Documents/autopolls/tflite_training/train_val_images/Insecta/'
#images_dir = '/content/drive/MyDrive/iNat/train_val_images/Insecta'

#%%
insectaAnnotations = []
imageIDs = np.array([ele['id'] for ele in annotations['images']])

plntCnt=0
for cnt,ele in enumerate(annotations['annotations']):
  ind = np.where(imageIDs == ele['image_id'])[0][0]
  tempAnnot = annotations['images'][ind]
  if tempAnnot['file_name'].split('/')[1] == 'Insecta':
    temp = {}
    for ele2 in tempAnnot:
      temp[ele2] = tempAnnot[ele2]
    for ele2 in ele:
      temp[ele2] = ele[ele2]
    insectaAnnotations.append(temp)
#%%
bees = []
for ele in insectaAnnotations:
  tempFile = ele['file_name'].split('/')[2].split(' ')[0]
  isF = np.nansum([ele2.find(tempFile) != -1 for ele2 in finalHyms])
  if isF>0:
    bees.append(ele)
#%%
fCf = {}
for ele in fCat:
  fCf[ele] = []

for ele in insectaAnnotations:
  tempFile = ele['file_name'].split('/')[2].split(' ')[0]
  for cat1 in fCat:
      isF = np.nansum([ele2.find(tempFile) != -1 for ele2 in classDic1[cat1]])
      if isF>0:
        fCf[cat1].append(ele)
        break


#%% FIND ANGIOSPERSM
in1 = open('/home/actmsofficial/Documents/annotations/iNat2021/train.json','r')
out = json.load(in1)
in1.close()
imNames = out['images']
finalPlants = []

for ele in imNames:
  if ele['file_name'].find('angiosperm') != -1:
    finalPlants.append(ele['file_name'])
#%% ADDING ANGIOSPERM FILTER FOR PLANTS ADDING TO MULTI CLASS
import glob
images_dir = glob.glob('/home/actmsofficial/Documents/autopolls/tflite_training/train_val_images/Plantae/*')
base = '/home/actmsofficial/Documents/autopolls/tflite_training/train_val_images/Plantae/'
#for instc in range(0,4000):
    #temp1 = np.int(np.floor(np.random.rand()*len(images_dir)))
plantAnnot = []
for temp1 in images_dir:
    tempBase = temp1+'/'
    tnameF = temp1.split('/')[-1].split(' ')[0]
    for ele in finalPlants:
      if ele.find(tnameF) != -1:
        lstFs = glob.glob(tempBase+'*')
        for ele2 in lstFs:
          temp = {}
          temp['file_name'] = ele2
          plantAnnot.append(temp)

#%%
fCf['Plant'] = []
ind = np.array(np.arange(0,len(plantAnnot)))
np.random.shuffle(ind)
ind2 = np.array(ind)[:7500]
#ind2 = np.array(ind)[7500:10500]
for ele in ind2:
  fCf['Plant'].append(plantAnnot[ele])

#%%

#ind = np.array(np.arange(0,len(insectaAnnotations)))
ind = np.array(np.arange(0,len(bees)))

np.random.shuffle(ind)

import csv
#ind = np.array(np.arange(0,len(insectaAnnotations)))
ind = np.array(np.arange(0,len(bees)))

np.random.shuffle(ind)
ind2 = np.array(ind)

with open('/home/actmsofficial/Desktop/trainDet_iNat_7class_train.csv','w',newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  #for ele in insectaAnnotations[ind2]:
  for eleInd in ind2:
    #ele = insectaAnnotations[eleInd]
    ele = bees[eleInd]
    if ele['file_name'].split('/')[1] == 'Insecta':
        xmin = ele['bbox'][0]/ele['width']
        ymin = ele['bbox'][1]/ele['height']
        xmax = xmin+ele['bbox'][2]/ele['width']
        ymax = ymin+ele['bbox'][3]/ele['height']

        tempName = ele['file_name'].split('/')[2]
        t1 = 'TRAIN'
        s1 = ele['file_name'].split('Insecta')[0]+'InsectaVal'+ele['file_name'].split('Insecta')[1]
        t2 = '/home/actmsofficial/Documents/autopolls/tflite_training/train_val_images/Insecta/'+s1.split('InsectaVal/')[1]
        t3 = 'insect'#tempName #tempName.split(' ')[0]+tempName.split(' ')[1]
        t4 = str(xmin)
        t5 = str(ymin)
        t6 = ''
        t7 = ''
        t8 = str(xmax)
        t9 = str(ymax)
        t10 = ''
        t11 = None
        #tempLine = ['test',None,'test2']
        spamwriter.writerow([t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11])
    elif ele['file_name'].split('/')[1] == 'Plantae':
        tempName = ele['file_name'].split('/')[2]
        t1 = 'TRAIN'
        t2 = ele['file_name']
        t3 = ''
        t4 = ''
        t5 = ''
        t6 = ''
        t7 = ''
        t8 = ''
        t9 = ''
        t10 = ''
        t11 = None
        #tempLine = ['test',None,'test2']
        spamwriter.writerow([t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11])


#%% MAKING CSV FOR MULTICLASS
#ind = np.array(np.arange(0,len(insectaAnnotations)))
#ind = np.array(np.arange(0,len(bees)))

#np.random.shuffle(ind)

import csv
#ind = np.array(np.arange(0,len(insectaAnnotations)))

with open('/home/actmsofficial/Desktop/trainDet_iNat_1Class_train_7classdata.csv','w',newline='') as csvfile:
  spamwriter = csv.writer(csvfile)
  #for ele in insectaAnnotations[ind2]:
  for knee in fCf:
    bees = fCf[knee]
    ind = np.array(np.arange(0,len(bees)))
    np.random.shuffle(ind)
    ind2 = np.array(ind)
    for eleInd in ind2:
      #ele = insectaAnnotations[eleInd]
      ele = bees[eleInd]
      if ele['file_name'].split('/')[1] == 'Insecta':
          xmin = ele['bbox'][0]/ele['width']
          ymin = ele['bbox'][1]/ele['height']
          xmax = xmin+ele['bbox'][2]/ele['width']
          ymax = ymin+ele['bbox'][3]/ele['height']

          tempName = ele['file_name'].split('/')[2]
          t1 = 'TRAIN'
          s1 = ele['file_name'].split('Insecta')[0]+'InsectaVal'+ele['file_name'].split('Insecta')[1]
          t2 = '/home/actmsofficial/Documents/autopolls/tflite_training/train_val_images/Insecta/'+s1.split('InsectaVal/')[1]
          t3 = 'Insecta'#knee#'insect'#tempName #tempName.split(' ')[0]+tempName.split(' ')[1]
          t4 = str(xmin)
          t5 = str(ymin)
          t6 = ''
          t7 = ''
          t8 = str(xmax)
          t9 = str(ymax)
          t10 = ''
          t11 = None
          #tempLine = ['test',None,'test2']
          spamwriter.writerow([t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11])
      elif ele['file_name'].split('/')[1] == 'Plantae':
          tempName = ele['file_name'].split('/')[2]
          t1 = 'TRAIN'
          t2 = ele['file_name']
          t3 = ''
          t4 = ''
          t5 = ''
          t6 = ''
          t7 = ''
          t8 = ''
          t9 = ''
          t10 = ''
          t11 = None
          #tempLine = ['test',None,'test2']
          spamwriter.writerow([t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11])

#%%

import numpy as np
import os
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
#%%
spec = object_detector.EfficientDetLite0Spec()
#%%
#spec = model_spec.get("efficientdet_lite0")
spec.config.autoaugment_policy = 'v3'
#%%
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('/home/actmsofficial/Desktop/trainDet_iNat_1Class_train_7classdata.csv')
validation_data, x, test_data = object_detector.DataLoader.from_csv('/home/actmsofficial/Desktop/trainDet_iNat_1Class_val_7classdata.csv')
#%%
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#%%
model = object_detector.create(train_data=train_data,
                               model_spec=spec,
                               epochs=20,
                               batch_size=4,
                               validation_data = validation_data,
                               train_whole_model=True)
# %%
TFLITE_FILENAME = 'efficientdet-lite_320x320_1Class_v2.tflite'
LABELS_FILENAME = 'efficientdet-lite_320x320_1Class_v1-labels.txt'
model.export(export_dir='/home/actmsofficial/Desktop/', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])
# %%


in1 = open('/home/actmsofficial/Desktop/trainDet_iNat_insect_wVal.csv','r')
in2 = in1.readlines()


#%%
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import tflite_runtime.interpreter as tflite
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    color = (250,50,120)
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=2)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

def make_interpreter(model_file):
  model_file, *device = model_file.split('@')
  return tflite.Interpreter(
      model_path=model_file,
      experimental_delegates=[
          tflite.load_delegate('libedgetpu.so.1',
                               {'device': device[0]} if device else {})
      ])

#%%
cnt = 0
#%%
import glob
# Load the TF Lite model
#baseF = '/home/actormsofficial/Documents/tflite_training/'
baseF = '/home/actmsofficial/Desktop/iNatBees_tflite/'
#TFLITE_FILENAME = '/home/actormsofficial/Desktop/efficientdet-lite_320x320_iNat_II.tflite'
#TFLITE_FILENAME = '/home/actormsofficial/Desktop/efficientdet-lite_320x320_iNat_insectDetect.tflite'
#labels = read_label_file('/home/actormsofficial/Desktop/efficientdet-lite_320x320_iNat_II-labels.txt')
#labels =  read_label_file('/home/actormsofficial/Desktop/efficientdet-lite_320x320_iNat_insectDetect-labels.txt')

#TFLITE_FILENAME = 'efficientdet-lite_320x320_iNatBees_insectDetect_wv.tflite'
#LABELS_FILENAME = 'efficientdet-lite_320x320_iNatBees_insectDetect_wv-labels.txt'
baseF = '/home/actormsofficial/Desktop/'
TFLITE_FILENAME = 'efficientdet-lite_320x320_1Class_v2.tflite'
LABELS_FILENAME = 'efficientdet-lite_320x320_1Class_v2-labels.txt'

labels = read_label_file(baseF+LABELS_FILENAME)
interpreter = tflite.Interpreter(baseF+TFLITE_FILENAME)
#interpreter = make_interpreter(TFLITE_FILENAME)
interpreter.allocate_tensors()

# Resize the image for input
temp1 = glob.glob('/home/actormsofficial/Documents/autopolls/tflite_training/train_val_images/Insecta/*')

r1 = np.int(np.floor(np.random.rand()*len(temp1)))
temp2 = temp1[r1]
temp3 = glob.glob(temp2+'/*')
r1 = np.int(np.floor(np.random.rand()*len(temp3)))
temp4 = temp3[r1]

#r1 = np.int(np.ceil(np.random.rand()*len(in2)))
#temp4 = in2[r1].split(',')[1]

#temp4 = '/home/actormsofficial/Downloads/0028.png'
#temp4 = '/home/actormsofficial/Downloads/2021-08-06_13-14-01000024.jpg'
#image = Image.open('/home/actormsofficial/Desktop/train_val_images/Insecta/Apis mellifera/0c170d60149fb8187ef972871400611d.jpg')
temp4 = '/home/actormsofficial/Documents/autopolls/vidFern2_exampleInference/241.jpg'
#temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/0054_05.43.46[R][0@0][0].jpg'
#temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/1528_14.44.44[R][0@0][0].jpg'
#temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/1575_05.58.34[R][0@0][0].jpg'
#temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/1590_17.00.40[R][0@0][0].jpg'
temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/1572_12.05.34[R][0@0][0].jpg'
#temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/1603_11.27.42[R][0@0][0].jpg'
temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/1610_17.21.41[R][0@0][0].jpg'
#temp4 = '/home/actormsofficial/Documents/autopolls/pollinatorCam2020_singleFlower/images/1609_11.02.12[R][0@0][0].jpg'
#temp4 = '/home/actormsofficial/Documents/annotations/apRetrain/stable/000003.png'
image = Image.open(temp4)
_, scale = common.set_resized_input(interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.25, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor,labels)
image.save('/home/actormsofficial/Desktop/%03d.png'%cnt)
image
cnt+=1
# %%

import cv2 as cv

vid1 = cv.VideoCapture('/home/actormsofficial/Downloads/Bombus_34.mp4')
readIn,image = vid1.read()
cnt = 0
while readIn:
  cv.imwrite('/home/actormsofficial/Documents/tempVideo/%03d.png'%cnt,image)
  cnt+=1
  readIn,image = vid1.read()


# %%
fList = glob.glob('/home/actormsofficial/Documents/tempVideo/*')
fList.sort()
cnt = 0
for temp4 in fList:
  image = Image.open(temp4)
  _, scale = common.set_resized_input(interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

  # Run inference
  interpreter.invoke()
  objs = detect.get_objects(interpreter, score_threshold=0.3, image_scale=scale)

  # Resize again to a reasonable size for display
  display_width = 2000
  scale_factor = display_width / image.width
  height_ratio = image.height / image.width
  image = image.resize((display_width, int(display_width * height_ratio)))
  draw_objects(ImageDraw.Draw(image), objs, scale_factor,labels)
  image.save('/home/actormsofficial/Documents/%03d.png'%cnt)
  cnt+=1
# %%
getF = glob.glob('/home/actormsofficial/Documents/ts/*')
getF.sort()
out = cv.VideoWriter('/home/actormsofficial/beeforage.avi',cv.VideoWriter_fourcc('M','J','P','G'),6,(2000,1500))
for ele in getF:
  in1 = cv.imread(ele)
  out.write(in1)
out.release()

# %%
